# Load the data

In [ ]:
## setup analysis
#execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
execfile(r'//Users/humphreys/Repositories/personal_calcs/setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'//Users/humphreys/Documents/teamdiamond/data/PurifyData/lt3/Purify_XX_First_Attempts'
folder_lt4 = r'//Users/humphreys/Documents/teamdiamond/data/PurifyData/lt4/Purify_XX_First_Attempts'

### ssro calibrations
day = '20160716_' # for ssro calibration

ssro_calib_lt3 = day+'091002'
ssro_calib_lt4 = day+'113952'

Purify_analysis = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)


In [ ]:

### list of good timestamps and RO calibration
all_lt3,all_lt4 = [],[]

### get all valid time stamps. remember that data has to be stored locally.
msmt_days  = ['20160716','20160714','20160715']
for d in msmt_days:

    tstamp_lt3,tstamp_lt4 = Purify_analysis.tstamps_for_both_setups(d,contains = 'ZZ')#,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)

    
print all_lt3
print all_lt4

In [ ]:
Purify_analysis.load_raw_data(all_lt3,all_lt4)
# BK_analysis.filter_CR_after(threshold =None)

In [ ]:
Purify_analysis.check_tail_w1_w2(st_start = 2774e3,st_len = 50e3)

In [ ]:
### autocorrelation of photon clicks --> do we have huge waiting times in the experiment
# reload(pq_plots)
# a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[0],folder = folder_lt4))
# print a_lt4
# pq_plots.plot_autocorrelation_histogram(a_lt4.pqf,start = 0,length = 50e3,binsize=0.1e3)

### Filtering and correlations

In [ ]:
photon_channel = 2 # 0 or 1; 2 means both HH detectors
st_start = 2774e3#2763e3
st_len       = 20e3 #50 ns
st_len_w2    = 20e3
ch1_offset = 0e3
max_reps_w2 = 30

In [ ]:
### this window can be used to analyze one of the full timetrace in order to get a feeling for the tail and the windows.
### later on all timestamps should be combined to get a clear pikkcture

a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[0],folder = folder_lt4))

vert0 = st_start/1000. ### start of the first window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =2765e3,  length = 80e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert0) #start =7480
# pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =1965e3+1000e3,  length = 10e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert1,plot_threshold_ch1 =vert1) #start =7480
# pq_plots.plot_photon_hist(b.pqf,start =1965+500,  length = 250, hist_binsize = 1, save = False, log=True) #start =7480

In [ ]:
### return the length of events without temporal filter:

no_of_events = 0
for i in range(len(Purify_analysis.lt3_dict['ssro_results'])):
    no_of_events += len(Purify_analysis.lt3_dict['ssro_results'][i])

print 'number of unfiltered (temporal and CR) events', no_of_events



### apply temporal filter (see above for filters)
#Purify_analysis.filter_CR_after(threshold = 1,verbose = False) ## NOT IMPLEMENTED
Purify_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                       st_len = st_len,
                                                       st_len_w2 = st_len_w2
                                                       )
# ch1_offset = ch1_offset NOT IMPLEMENTED

In [ ]:
Purify_analysis.apply_sync_filter_w1_w2(verbose = True,max_w2 = max_reps_w2)
Purify_analysis.apply_is_purified_filter(signature = '11', verbose = True)

In [ ]:
Purify_analysis.apply_CR_before_filter(verbose = True)
Purify_analysis.apply_CR_after_filter(verbose = True)

In [ ]:
Purify_analysis.attach_state_filtered_syncs(verbose = False)

In [ ]:
### applies the time fitlered sync numbers to the adwin RO results of each setup
Purify_analysis.correlate_RO_results(verbose = True,apply_ROC = True)

In [ ]:
### correlations vs. window start?
Purify_analysis.sweep_filter_parameter_vs_correlations('st_start',np.linspace(2772e3,2776e3,20),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('st_len',np.linspace(2e3,50e3,30),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('max_reps_w2',np.arange(15,250,10),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('max_dt',np.linspace(1e3,30e3,20),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt3_before',np.linspace(20,40,20),apply_ROC = True)
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt4_before',np.linspace(20,40,20),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt3_after',np.linspace(10,30,20),apply_ROC = True)
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt4_after',np.linspace(10,30,20),apply_ROC = True)

In [ ]:
#### put readout corrected analysis here.